**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../config/config_sing.R")))
show_env()

You are in Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out 
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC 
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc 
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log 


In [2]:
fdiry = file.path(FD_RES, "results", "region")
dir(fdiry)

[1] "annotation_ccre"                        
 [2] "annotation_chipseq_histone"             
 [3] "annotation_chipseq_tf"                  
 [4] "annotation_chipseq_tf_subset"           
 [5] "annotation_chromHMM"                    
 [6] "annotation_crispri_e2g"                 
 [7] "annotation_crispri_growth"              
 [8] "annotation_crispri_hcrff"               
 [9] "annotation_enhancer_junke"              
[10] "annotation_enhancer_merge"              
[11] "annotation_enhancer_zscore_junke"       
[12] "annotation_gencode"                     
[13] "annotation_hic"                         
[14] "annotation_tss_pol2"                    
[15] "annotation_tss_pol2_span"               
[16] "hic_intact_deep"                        
[17] "hic_intact_ENCSR479XDG"                 
[18] "KS91_K562_ASTARRseq_peak_macs_input"    
[19] "region_screened_crispri_flowfish.bed"   
[20] "region_screened_selected_perbase.bed"   
[21] "region_screened_selected_perbase.bed.gz"
[22] "region_screened_selected.bed"           
[23] "region_screened_tmpra_crispr.bed"       
[24] "region_screened_tmpra.bed"

## Import data

In [3]:
FOLDER_ANT   = "KS91_K562_ASTARRseq_peak_macs_input"
FOLDERS_LOOP = c(
    "hic_intact_ENCSR479XDG",
    "hic_intact_deep"
)
PREFIXS_LOOP = c("loopA", "loopB")

### Import loop data

In [4]:
lst = lapply(FOLDERS_LOOP, function(FOLDER_LOOP){
    fdiry = file.path(FD_RES, "results", "region", FOLDER_LOOP, "loop")
    fname = "loop.tsv"
    fpath = file.path(fdiry, fname)
    
    dat = read_tsv(fpath, show_col_types = FALSE)
    dat = dat %>% dplyr::mutate(
        HiC = FOLDER_LOOP,
        Edge = paste(LoopA, LoopB, sep="|")
    )
    return(dat)
})

dat = bind_rows(lst)
dat_edge = dat
lst_edge = split(dat, dat$HiC)
print(dim(dat))
head(dat)

[1] 174718     13


Chrom1,Start1,End1,Chrom2,Start2,End2,LoopA,LoopB,LengthA,LengthB,Distance,HiC,Edge
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
chr10,102835000,102836000,chr10,102901000,102902000,chr10:102835000-102836000,chr10:102901000-102902000,1000,1000,66000,hic_intact_ENCSR479XDG,chr10:102835000-102836000|chr10:102901000-102902000
chr10,123583000,123584000,chr10,123967000,123968000,chr10:123583000-123584000,chr10:123967000-123968000,1000,1000,384000,hic_intact_ENCSR479XDG,chr10:123583000-123584000|chr10:123967000-123968000
chr10,60780000,60782000,chr10,60828000,60830000,chr10:60780000-60782000,chr10:60828000-60830000,2000,2000,48000,hic_intact_ENCSR479XDG,chr10:60780000-60782000|chr10:60828000-60830000
chr10,33050000,33051000,chr10,33067000,33068000,chr10:33050000-33051000,chr10:33067000-33068000,1000,1000,17000,hic_intact_ENCSR479XDG,chr10:33050000-33051000|chr10:33067000-33068000
chr10,11412000,11414000,chr10,11472000,11474000,chr10:11412000-11414000,chr10:11472000-11474000,2000,2000,60000,hic_intact_ENCSR479XDG,chr10:11412000-11414000|chr10:11472000-11474000
chr10,45005000,45010000,chr10,45465000,45470000,chr10:45005000-45010000,chr10:45465000-45470000,5000,5000,460000,hic_intact_ENCSR479XDG,chr10:45005000-45010000|chr10:45465000-45470000


In [5]:
print(names(lst_edge))

[1] "hic_intact_deep"        "hic_intact_ENCSR479XDG"


In [6]:
## import annotation description file
fdiry  = file.path(FD_RES, "results", "region", FOLDER_ANT)
fname = "description.tsv"
fpath = file.path(fdiry, fname)
dat = read_tsv(fpath, show_col_types = FALSE)
cnames = dat$Name
cat(FOLDER_ANT, "\n")
print(cnames)
cat("\n")

### get file path of annotation file
fdiry = file.path(FD_RES, "results", "region", FOLDER_ANT)
fname = "*.bed.gz"
fglob = file.path(fdiry, fname)
fpath_ant = Sys.glob(fglob)
print(fpath_ant)
cat("\n")

KS91_K562_ASTARRseq_peak_macs_input 
[1] "Chrom" "Start" "End"  

[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed.gz"



### Import annotation data

In [7]:
### init
fname_ant = basename(fpath_ant)

### import annotation file
dat_ant = read_tsv(fpath_ant, col_names = cnames, show_col_types = FALSE)
dat_ant = dat_ant %>% dplyr::mutate(Region_Annot = paste0(Chrom, ":", Start, "-", End))
head(dat_ant)

Chrom,Start,End,Region_Annot
<chr>,<dbl>,<dbl>,<chr>
chr1,10015,10442,chr1:10015-10442
chr1,14253,14645,chr1:14253-14645
chr1,16015,16477,chr1:16015-16477
chr1,17237,17772,chr1:17237-17772
chr1,28903,29613,chr1:28903-29613
chr1,30803,31072,chr1:30803-31072


In [18]:
lst = lapply(FOLDERS_LOOP, function(FOLDER_LOOP){

    ### get file path for corresponding loop annotation
    fdiry = file.path(FD_RES, "results", "region", FOLDER_LOOP, "loop", FOLDER_ANT)
    
    lst = lapply(PREFIXS_LOOP, function(PREFIX_LOOP){
        
        ###
        fname = paste0(PREFIX_LOOP, "*", fname_ant)
        fglob = file.path(fdiry, fname)
        fpath = Sys.glob(fglob)
        
        ###
        vec = c("Chrom_Loop", "Start_Loop", "End_Loop", cnames, "Overlap")
        dat = read_tsv(fpath, col_names = vec, show_col_types = FALSE)
        dat = dat %>% dplyr::mutate(
            Region_Annot = paste0(Chrom, ":", Start, "-", End),
            Region_Loop  = paste0(Chrom_Loop, ":", Start_Loop, "-", End_Loop),
            Loop         = PREFIX_LOOP
        )
        return(dat)
    })
    dat = bind_rows(lst)
    dat = dat %>% dplyr::mutate(HiC = FOLDER_LOOP)
    return(dat)
})

###
dat = bind_rows(lst)
dat = dat %>% 
    dplyr::mutate(Label = paste(HiC, Loop, sep=".")) %>%
    dplyr::select(Label, Region_Loop, Region_Annot) %>%
    dplyr::distinct()

### assign and show
dat_loop_ocr = dat
lst_loop_ocr = split(dat, dat$Label)
print(dim(dat))
head(dat)

[1] 180110      3


Label,Region_Loop,Region_Annot
<chr>,<chr>,<chr>
hic_intact_ENCSR479XDG.loopA,chr1:770000-780000,chr1:770944-771343
hic_intact_ENCSR479XDG.loopA,chr1:770000-780000,chr1:777949-779437
hic_intact_ENCSR479XDG.loopA,chr1:904000-906000,chr1:903633-905556
hic_intact_ENCSR479XDG.loopA,chr1:920000-925000,chr1:920697-921526
hic_intact_ENCSR479XDG.loopA,chr1:920000-925000,chr1:923417-925194
hic_intact_ENCSR479XDG.loopA,chr1:980000-990000,chr1:984016-984547


In [19]:
dat = dat_loop_ocr
unique(dat$Label)

[1] "hic_intact_ENCSR479XDG.loopA" "hic_intact_ENCSR479XDG.loopB"
[3] "hic_intact_deep.loopA"        "hic_intact_deep.loopB"

In [20]:
lst = lst_loop_ocr
print(names(lst))

[1] "hic_intact_deep.loopA"        "hic_intact_deep.loopB"       
[3] "hic_intact_ENCSR479XDG.loopA" "hic_intact_ENCSR479XDG.loopB"


## Connect OCRs by loops

**hic_intact_ENCSR479XDG**

In [63]:
###
lst = lst_edge
dat_loop = lst[["hic_intact_ENCSR479XDG"]]

lst = lst_loop_ocr
dat_loopA = lst[["hic_intact_ENCSR479XDG.loopA"]]
dat_loopB = lst[["hic_intact_ENCSR479XDG.loopB"]]

###
dat_loopA = dat_loopA %>% 
    dplyr::rename(
        LabelA = Label,
        LoopA  = Region_Loop,
        NodeA  = Region_Annot
    )

dat_loopB = dat_loopB %>% 
    dplyr::rename(
        LabelB = Label,
        LoopB  = Region_Loop,
        NodeB  = Region_Annot
    )

###
dat_edge = dat_loop
dat_edge = dat_edge %>% dplyr::left_join(dat_loopA, by = "LoopA", relationship = "many-to-many") %>% na.omit
dat_edge = dat_edge %>% dplyr::left_join(dat_loopB, by = "LoopB", relationship = "many-to-many") %>% na.omit
dat_node = data.frame(Node = unique(c(dat_edge$NodeA, dat_edge$NodeB)))

### assign and show
cat("Check Edge Label A:", unique(dat_edge$LabelA), "\n")
cat("Check Edge Label B:", unique(dat_edge$LabelB), "\n")
cat("#Loop (Total):     ", nrow(dat_loop), "\n")
cat("#Loop (Total):     ", length(unique(dat_loop$Edge)), "\n")
cat("#Link (OCR):       ", nrow(dat_edge), "\n")
cat("#Edge (OCR):       ", length(unique(dat_edge$Edge)), "\n")
cat("#Node (OCR):       ", nrow(dat_node), "\n")

dat_node_hic_intact_ENCSR479XDG = dat_node
dat_edge_hic_intact_ENCSR479XDG = dat_edge
head(dat_edge)

Check Edge Label A: hic_intact_ENCSR479XDG.loopA 
Check Edge Label B: hic_intact_ENCSR479XDG.loopB 
#Loop (Total):      46004 
#Loop (Total):      46004 
#Link (OCR):        53964 
#Edge (OCR):        26619 
#Node (OCR):        42110 


Chrom1,Start1,End1,Chrom2,Start2,End2,LoopA,LoopB,LengthA,LengthB,Distance,HiC,Edge,LabelA,NodeA,LabelB,NodeB
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
chr10,11412000,11414000,chr10,11472000,11474000,chr10:11412000-11414000,chr10:11472000-11474000,2000,2000,60000,hic_intact_ENCSR479XDG,chr10:11412000-11414000|chr10:11472000-11474000,hic_intact_ENCSR479XDG.loopA,chr10:11412050-11413031,hic_intact_ENCSR479XDG.loopB,chr10:11471940-11472592
chr10,11412000,11414000,chr10,11472000,11474000,chr10:11412000-11414000,chr10:11472000-11474000,2000,2000,60000,hic_intact_ENCSR479XDG,chr10:11412000-11414000|chr10:11472000-11474000,hic_intact_ENCSR479XDG.loopA,chr10:11412050-11413031,hic_intact_ENCSR479XDG.loopB,chr10:11473227-11473648
chr10,120420000,120430000,chr10,120600000,120610000,chr10:120420000-120430000,chr10:120600000-120610000,10000,10000,180000,hic_intact_ENCSR479XDG,chr10:120420000-120430000|chr10:120600000-120610000,hic_intact_ENCSR479XDG.loopA,chr10:120423274-120423638,hic_intact_ENCSR479XDG.loopB,chr10:120600849-120601348
chr10,67885000,67890000,chr10,69315000,69320000,chr10:67885000-67890000,chr10:69315000-69320000,5000,5000,1430000,hic_intact_ENCSR479XDG,chr10:67885000-67890000|chr10:69315000-69320000,hic_intact_ENCSR479XDG.loopA,chr10:67885989-67886387,hic_intact_ENCSR479XDG.loopB,chr10:69315429-69316171
chr10,67885000,67890000,chr10,69315000,69320000,chr10:67885000-67890000,chr10:69315000-69320000,5000,5000,1430000,hic_intact_ENCSR479XDG,chr10:67885000-67890000|chr10:69315000-69320000,hic_intact_ENCSR479XDG.loopA,chr10:67885989-67886387,hic_intact_ENCSR479XDG.loopB,chr10:69317164-69319347
chr10,110300000,110310000,chr10,112310000,112320000,chr10:110300000-110310000,chr10:112310000-112320000,10000,10000,2010000,hic_intact_ENCSR479XDG,chr10:110300000-110310000|chr10:112310000-112320000,hic_intact_ENCSR479XDG.loopA,chr10:110304321-110305606,hic_intact_ENCSR479XDG.loopB,chr10:112313241-112313740


**hic_intact_deep**

In [64]:
###
lst = lst_edge
dat_loop = lst[["hic_intact_deep"]]

lst = lst_loop_ocr
dat_loopA = lst[["hic_intact_deep.loopA"]]
dat_loopB = lst[["hic_intact_deep.loopB"]]

###
dat_loopA = dat_loopA %>% 
    dplyr::rename(
        LabelA = Label,
        LoopA  = Region_Loop,
        NodeA  = Region_Annot
    )

dat_loopB = dat_loopB %>% 
    dplyr::rename(
        LabelB = Label,
        LoopB  = Region_Loop,
        NodeB  = Region_Annot
    )

###
dat_edge = dat_loop
dat_edge = dat_edge %>% dplyr::left_join(dat_loopA, by = "LoopA", relationship = "many-to-many") %>% na.omit
dat_edge = dat_edge %>% dplyr::left_join(dat_loopB, by = "LoopB", relationship = "many-to-many") %>% na.omit
dat_node = data.frame(Node = unique(c(dat_edge$NodeA, dat_edge$NodeB)))

### assign and show
cat("Check Edge Label A:", unique(dat_edge$LabelA), "\n")
cat("Check Edge Label B:", unique(dat_edge$LabelB), "\n")
cat("#Loop (Total):     ", nrow(dat_loop), "\n")
cat("#Loop (Total):     ", length(unique(dat_loop$Edge)), "\n")
cat("#Link (OCR):       ", nrow(dat_edge), "\n")
cat("#Edge (OCR):       ", length(unique(dat_edge$Edge)), "\n")
cat("#Node (OCR):       ", nrow(dat_node), "\n")

### assign and show
dat_node_hic_intact_deep = dat_node
dat_edge_hic_intact_deep = dat_edge
head(dat_edge)

Check Edge Label A: hic_intact_deep.loopA 
Check Edge Label B: hic_intact_deep.loopB 
#Loop (Total):      128714 
#Loop (Total):      128714 
#Link (OCR):        84150 
#Edge (OCR):        61832 
#Node (OCR):        49673 


Chrom1,Start1,End1,Chrom2,Start2,End2,LoopA,LoopB,LengthA,LengthB,Distance,HiC,Edge,LabelA,NodeA,LabelB,NodeB
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
chr10,72223000,72224000,chr10,72328000,72329000,chr10:72223000-72224000,chr10:72328000-72329000,1000,1000,105000,hic_intact_deep,chr10:72223000-72224000|chr10:72328000-72329000,hic_intact_deep.loopA,chr10:72222974-72223526,hic_intact_deep.loopB,chr10:72328024-72329889
chr10,64041000,64042000,chr10,67697000,67698000,chr10:64041000-64042000,chr10:67697000-67698000,1000,1000,3656000,hic_intact_deep,chr10:64041000-64042000|chr10:67697000-67698000,hic_intact_deep.loopA,chr10:64041063-64042467,hic_intact_deep.loopB,chr10:67697784-67698056
chr10,78302000,78304000,chr10,79194000,79196000,chr10:78302000-78304000,chr10:79194000-79196000,2000,2000,892000,hic_intact_deep,chr10:78302000-78304000|chr10:79194000-79196000,hic_intact_deep.loopA,chr10:78302868-78304016,hic_intact_deep.loopB,chr10:79195028-79196033
chr10,94362000,94363000,chr10,94444000,94445000,chr10:94362000-94363000,chr10:94444000-94445000,1000,1000,82000,hic_intact_deep,chr10:94362000-94363000|chr10:94444000-94445000,hic_intact_deep.loopA,chr10:94362154-94363449,hic_intact_deep.loopB,chr10:94444393-94444687
chr10,11780000,11785000,chr10,12710000,12715000,chr10:11780000-11785000,chr10:12710000-12715000,5000,5000,930000,hic_intact_deep,chr10:11780000-11785000|chr10:12710000-12715000,hic_intact_deep.loopA,chr10:11780611-11781410,hic_intact_deep.loopB,chr10:12710834-12711579
chr10,11780000,11785000,chr10,12710000,12715000,chr10:11780000-11785000,chr10:12710000-12715000,5000,5000,930000,hic_intact_deep,chr10:11780000-11785000|chr10:12710000-12715000,hic_intact_deep.loopA,chr10:11780611-11781410,hic_intact_deep.loopB,chr10:12712881-12713104


**Combine**

In [65]:
lst_dat_node = list(
    "hic_intact_ENCSR479XDG" = dat_node_hic_intact_ENCSR479XDG,
    "hic_intact_deep"        = dat_node_hic_intact_deep
)

lst_dat_edge = list(
    "hic_intact_ENCSR479XDG" = dat_edge_hic_intact_ENCSR479XDG,
    "hic_intact_deep"        = dat_edge_hic_intact_deep
)

## Arrange data

In [81]:
head(dat_node_hic_intact_ENCSR479XDG)

,Node
,<chr>
1,chr10:11412050-11413031
2,chr10:120423274-120423638
3,chr10:67885989-67886387
4,chr10:110304321-110305606
5,chr10:110306041-110306427
6,chr10:14837541-14839476


In [91]:
lst = lst_dat_node
lst = lapply(lst, function(dat){
    dat = dat %>% 
        tidyr::separate(Node, c("Chrom", "Start", "End"), remove = FALSE) %>%
        dplyr::distinct() %>%
        dplyr::arrange(Chrom, Start, End)
    return(dat)
})

lst_dat_node_arrange = lst
print(names(lst))
print(dim(lst[[1]]))
print(dim(lst[[2]]))

[1] "hic_intact_ENCSR479XDG" "hic_intact_deep"       
[1] 42110     4
[1] 49673     4


In [86]:
lst = lst_dat_node_arrange
head(lst[[1]], 3)

,Node,Chrom,Start,End
,<chr>,<chr>,<chr>,<chr>
1,chr1:100036871-100039191,chr1,100036871,100039191
2,chr1:100044617-100044899,chr1,100044617,100044899
3,chr1:100046250-100046578,chr1,100046250,100046578


In [87]:
lst = lst_dat_node_arrange
head(lst[[2]], 3)

,Node,Chrom,Start,End
,<chr>,<chr>,<chr>,<chr>
1,chr1:100036871-100039191,chr1,100036871,100039191
2,chr1:100071829-100072904,chr1,100071829,100072904
3,chr1:100074313-100074743,chr1,100074313,100074743


In [88]:
lst = lst_dat_edge
lst = lapply(lst, function(dat){
    vec = c(
        "NodeA",  "NodeB",   "Edge",
        "Chrom1",  "Start1", "End1", 
        "Chrom2",  "Start2", "End2", 
        "LoopA",   "LoopB", 
        "LengthA", "LengthB",
        "Distance",
        "HiC"
    )
    
    dat = dat %>% 
        dplyr::select({{vec}}) %>% 
        dplyr::arrange(Chrom1, Start1, End1, Chrom2, Start2, End2)
    
    return(dat)
})

lst_dat_edge_arrange = lst
print(names(lst))
print(dim(lst[[1]]))
print(dim(lst[[2]]))

[1] "hic_intact_ENCSR479XDG" "hic_intact_deep"       
[1] 53964    15
[1] 84150    15


In [89]:
lst = lst_dat_edge_arrange
head(lst[[1]], 3)

NodeA,NodeB,Edge,Chrom1,Start1,End1,Chrom2,Start2,End2,LoopA,LoopB,LengthA,LengthB,Distance,HiC
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
chr1:770944-771343,chr1:841328-842007,chr1:770000-780000|chr1:840000-850000,chr1,770000,780000,chr1,840000,850000,chr1:770000-780000,chr1:840000-850000,10000,10000,70000,hic_intact_ENCSR479XDG
chr1:770944-771343,chr1:842480-843449,chr1:770000-780000|chr1:840000-850000,chr1,770000,780000,chr1,840000,850000,chr1:770000-780000,chr1:840000-850000,10000,10000,70000,hic_intact_ENCSR479XDG
chr1:777949-779437,chr1:841328-842007,chr1:770000-780000|chr1:840000-850000,chr1,770000,780000,chr1,840000,850000,chr1:770000-780000,chr1:840000-850000,10000,10000,70000,hic_intact_ENCSR479XDG


In [90]:
lst = lst_dat_edge_arrange
head(lst[[2]], 3)

NodeA,NodeB,Edge,Chrom1,Start1,End1,Chrom2,Start2,End2,LoopA,LoopB,LengthA,LengthB,Distance,HiC
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
chr1:777949-779437,chr1:826754-828040,chr1:778000-779000|chr1:827000-828000,chr1,778000,779000,chr1,827000,828000,chr1:778000-779000,chr1:827000-828000,1000,1000,49000,hic_intact_deep
chr1:873248-873781,chr1:935605-936252,chr1:870000-875000|chr1:935000-940000,chr1,870000,875000,chr1,935000,940000,chr1:870000-875000,chr1:935000-940000,5000,5000,65000,hic_intact_deep
chr1:873248-873781,chr1:938228-938505,chr1:870000-875000|chr1:935000-940000,chr1,870000,875000,chr1,935000,940000,chr1:870000-875000,chr1:935000-940000,5000,5000,65000,hic_intact_deep


## Simplify the graph

In [77]:
lst = lst_dat_edge_arrange
lst = lapply(lst, function(dat){
    dat = dat %>%
        dplyr::group_by(NodeA, NodeB) %>%
        dplyr::summarise(
            Distance  = mean(Distance),
            Edges     = paste(Edge, collapse=","),
            Count     = n(),
            .groups   = "drop"
        )
    return(dat)
})

lst_dat_edge_simplify = lst
print(names(lst))

[1] "hic_intact_ENCSR479XDG" "hic_intact_deep"       


In [78]:
lst = lst_dat_edge_arrange
print(dim(lst[[1]]))
print(dim(lst[[2]]))

[1] 53964    15
[1] 84150    15


In [79]:
lst = lst_dat_edge_simplify
print(dim(lst[[1]]))
print(dim(lst[[2]]))

[1] 53963     5
[1] 84150     5


In [80]:
lst = lst_dat_edge_simplify
cat(names(lst)[1], "\n")
dat = lst[[1]]
print(head(dat))
print(table(dat$Count))
cat("\n")

cat(names(lst)[2], "\n")
dat = lst[[2]]
print(head(dat))
print(table(dat$Count))

hic_intact_ENCSR479XDG 
# A tibble: 6 × 5
  NodeA                     NodeB                     Distance Edges       Count
  <chr>                     <chr>                        <dbl> <chr>       <int>
1 chr10:100342247-100342824 chr10:100508470-100509309   166000 chr10:1003…     1
2 chr10:100342247-100342824 chr10:100871690-100872646   530000 chr10:1003…     1
3 chr10:100342247-100342824 chr10:100878006-100879519   530000 chr10:1003…     1
4 chr10:100346328-100347693 chr10:100871690-100872646   530000 chr10:1003…     1
5 chr10:100346328-100347693 chr10:100878006-100879519   530000 chr10:1003…     1
6 chr10:100358645-100359733 chr10:100373060-100374238    14000 chr10:1003…     1

    1     2 
53962     1 

hic_intact_deep 
# A tibble: 6 × 5
  NodeA                     NodeB                     Distance Edges       Count
  <chr>                     <chr>                        <dbl> <chr>       <int>
1 chr10:100045961-100046653 chr10:100433434-100434444   385000 chr10:1000…     1
2 ch

## Save results

In [94]:
print(FOLDER_ANT)
print(FOLDERS_LOOP)

[1] "KS91_K562_ASTARRseq_peak_macs_input"
[1] "hic_intact_ENCSR479XDG" "hic_intact_deep"       


In [92]:
lst = lst_dat_node_arrange
vec = names(lst)
print(vec)

[1] "hic_intact_ENCSR479XDG" "hic_intact_deep"       


In [93]:
lst = lst_dat_edge_simplify
vec = names(lst)
print(vec)

[1] "hic_intact_ENCSR479XDG" "hic_intact_deep"       


In [97]:
for (FOLDER_LOOP in FOLDERS_LOOP){
    ###
    fdiry = file.path(FD_RES, "results", "region", FOLDER_LOOP, "loop", FOLDER_ANT)
    print(fdiry)
    
    ###
    fname = "network_ocr_node.tsv"
    fpath = file.path(fdiry, fname)
    lst = lst_dat_node_arrange
    dat = lst[[FOLDER_LOOP]]
    write_tsv(dat, fpath)
    
    ###
    fname = "network_ocr_edge_simplify.tsv"
    fpath = file.path(fdiry, fname)
    lst = lst_dat_edge_simplify
    dat = lst[[FOLDER_LOOP]]
    write_tsv(dat, fpath)
}

[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/hic_intact_ENCSR479XDG/loop/KS91_K562_ASTARRseq_peak_macs_input"
[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/hic_intact_deep/loop/KS91_K562_ASTARRseq_peak_macs_input"
